# Fiftyville
Write SQL queries to solve a mystery.

## A Mystery in Fiftyville

The CS50 Duck has been stolen! The town of Fiftyville has called upon you to solve the mystery of the stolen duck. Authorities believe that the thief stole the duck and then, shortly afterwards, took a flight out of town with the help of an accomplice. Your goal is to identify:

Who the thief is,
What city the thief escaped to, and
Who the thief’s accomplice is who helped them escape
All you know is that the theft **took place on July 28, 2020** and that it **took place on Chamberlin Street**.

How will you go about solving this mystery? The Fiftyville authorities have taken some of the town’s records from around the time of the theft and prepared a SQLite database for you, fiftyville.db, which contains tables of data from around the town. You can query that table using SQL SELECT queries to access the data of interest to you. Using just the information in the database, your task is to solve the mystery.

Let's connect to database with sqlite3

In [1]:
from cs50 import SQL
database = SQL("sqlite:///fiftyville.db")

## SCENE DESCRIPTION

## 3 INTERVIEWS

👩‍🦳 RUTH: Sometime within ten minutes of the theft, I saw the thief get into a car in the courthouse parking lot and drive away.
If you have security footage from the courthouse parking lot, you might want to look for cars that left the parking lot in that time frame.

👱EUGENE: I don't know the thief's name, but it was someone I recognized. Earlier this morning, before I arrived at the courthouse,
I was walking by the ATM on Fifer Street and saw the thief there withdrawing some money.

🧓 RAYMOND: As the thief was leaving the courthouse, they called someone who talked to them for less than a minute.
In the call, I heard the thief say that they were planning to take the earliest flight out of Fiftyville tomorrow. The thief then asked the person on the other end of the phone to purchase the flight ticket.

## 1. PARKING LOT INFO
### 1.1 Which Licence left the courthouse between 10 & 11 am ?

In [2]:
# Take all of the licence who left the courthouse parking between 10 & 11 am
database.execute(
    '''
    SELECT license_plate 
    FROM courthouse_security_logs  
    WHERE 
        year = 2020 AND 
        month = 07 AND day =28 AND 
        activity = "exit" AND 
        hour >= 10  AND hour < 11;
    '''
)

[{'license_plate': '5P2BI95'},
 {'license_plate': '94KL13X'},
 {'license_plate': '6P58WS2'},
 {'license_plate': '4328GD8'},
 {'license_plate': 'G412CB7'},
 {'license_plate': 'L93JTIZ'},
 {'license_plate': '322W7JE'},
 {'license_plate': '0NTHK55'},
 {'license_plate': '1106N58'}]

### 1.2 Get the name of the potential suspect form there license_plate

In [3]:
### 1.2 Get the name of the potential suspect form there license_plate
database.execute('''
SELECT name FROM people WHERE license_plate IN
    (SELECT license_plate FROM courthouse_security_logs WHERE year = 2020 AND month = 07 AND day =28 AND activity = "exit" AND hour >= 10  AND hour < 11 AND minute<25);
    '''
)

[{'name': 'Patrick'},
 {'name': 'Amber'},
 {'name': 'Elizabeth'},
 {'name': 'Roger'},
 {'name': 'Danielle'},
 {'name': 'Russell'},
 {'name': 'Evelyn'},
 {'name': 'Ernest'}]

## 2. ATM INFO
### 2.1 : Get the account_number of people which withdraw money on Fifer street these day

In [4]:
database.execute('''
    SELECT account_number  
    FROM atm_transactions
    WHERE year = 2020 AND month = 07 AND day =28 AND transaction_type='withdraw' AND atm_location = "Fifer Street";
    '''
)

[{'account_number': 28500762},
 {'account_number': 28296815},
 {'account_number': 76054385},
 {'account_number': 49610011},
 {'account_number': 16153065},
 {'account_number': 25506511},
 {'account_number': 81061156},
 {'account_number': 26013199}]

### 2.2 GET the name of potential suspect with a request on atm_transaction then account_number then people

In [5]:
database.execute(
    '''
    SELECT name FROM people WHERE id IN
    (SELECT person_id FROM bank_accounts WHERE account_number IN
        (SELECT account_number  FROM atm_transactions WHERE year = 2020 AND month = 07 AND day =28 AND transaction_type='withdraw' AND atm_location = "Fifer Street"));
    '''
)

[{'name': 'Bobby'},
 {'name': 'Elizabeth'},
 {'name': 'Victoria'},
 {'name': 'Madison'},
 {'name': 'Roy'},
 {'name': 'Danielle'},
 {'name': 'Russell'},
 {'name': 'Ernest'}]

## 3 . FLIGHT INFO
Get the first flight from Fiftyville for the 29th Jully 2020

In [6]:
database.execute(
    '''
    SELECT people.name FROM passengers
    JOIN people ON people.passport_number = passengers.passport_number
    WHERE flight_id =
        (
            SELECT flights.id FROM flights
            JOIN airports ON flights.origin_airport_id = airports.id
            WHERE year = 2020 AND month = 07 AND day =28 AND city = "Fiftyville"
            ORDER BY hour
            LIMIT 1
        );
    '''
)

[{'name': 'Nicole'},
 {'name': 'Amanda'},
 {'name': 'Joyce'},
 {'name': 'Jean'},
 {'name': 'Daniel'},
 {'name': 'Carol'},
 {'name': 'Rebecca'},
 {'name': 'Sophia'}]

## 4. PHONE CALL
call with a duration under a minute

In [7]:
database.execute(
    '''
    SELECT name FROM people 
    WHERE phone_number IN (
        SELECT caller 
        FROM phone_calls 
        WHERE  
            year = 2020 AND month = 07 AND day = 28 AND DURATION < 60);
    '''
)

[{'name': 'Bobby'},
 {'name': 'Roger'},
 {'name': 'Victoria'},
 {'name': 'Madison'},
 {'name': 'Russell'},
 {'name': 'Evelyn'},
 {'name': 'Ernest'},
 {'name': 'Kimberly'}]

## Result 
### 1. Who is the thief ?

In [8]:
### Group everything in one querry
database.execute(
    '''
    SELECT name FROM people WHERE name IN (
    -- 1. PARKING LOT
    SELECT name FROM people WHERE license_plate IN(
        SELECT license_plate FROM courthouse_security_logs WHERE year=2020 AND month=07 AND day=28 AND activity = "exit" AND hour>=10  AND hour<11 AND minute<25
    )
)
AND name IN (
    -- 2 ATM
    SELECT name FROM people WHERE id IN(
        SELECT person_id FROM bank_accounts WHERE account_number IN(
            SELECT account_number  FROM atm_transactions WHERE year = 2020 AND month = 07 AND day =28 AND transaction_type='withdraw' AND atm_location = "Fifer Street"
        )
    )
)
AND name IN (
    -- 3 FLIGHT
    SELECT people.name FROM passengers
    JOIN people ON people.passport_number = passengers.passport_number
    WHERE flight_id =
        (
            SELECT flights.id FROM flights
            JOIN airports ON flights.origin_airport_id = airports.id
            WHERE year = 2020 AND month = 07 AND day = 29 AND city = "Fiftyville"
            ORDER BY hour
            LIMIT 1
        )
)
AND name IN (
    -- PHONE CALL
    SELECT name FROM people WHERE phone_number IN (
        SELECT caller FROM phone_calls WHERE  year = 2020 AND month = 07 AND day = 28 AND duration < 60
    )
);
    ''')

[{'name': 'Ernest'}]

Ernest is the thief 🥷

### Who is the helper ?

In [9]:
database.execute(
    '''
    SELECT name 
    FROM people WHERE phone_number = (
        SELECT phone_calls.receiver 
        FROM phone_calls
        JOIN people ON people.phone_number = phone_calls.caller
        WHERE  year = 2020 AND month = 07 AND day = 28 AND duration < 60 AND name = "Ernest");
    '''
)

[{'name': 'Berthold'}]

BERTHOLD IS THE HELPER 😈

### Where are they escaping to ?

In [10]:
database.execute(
    '''
    SELECT * 
    FROM airports 
    WHERE id =
        (
            SELECT destination_airport_id FROM flights
            JOIN airports ON flights.origin_airport_id = airports.id
            WHERE year = 2020 AND month = 07 AND day = 29 AND city = "Fiftyville"
            ORDER BY hour LIMIT 1
        );
    '''
)

[{'id': 4,
  'abbreviation': 'LHR',
  'full_name': 'Heathrow Airport',
  'city': 'London'}]

💂‍♀️ THEY ESCAPED TO LONDON


summary: 
# Ernest is the thief he escaped to London with Berthold 🥷💂‍♀️😈